In [1]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
from gensim.models import CoherenceModel
import tweepy
import pandas as pd
import numpy as np
import time

'''
抓取tweets：包括company_twitter_name list中的22家cultured meat的company，抓取過程中通過include_rts=False過濾掉retweets，
抓取後通過tweet.in_reply_to_status_id == None過濾掉所有reply
'''
consumer_key = 'YRWB6yizS63ZJUW2CpEUqS7TN'
consumer_secret = 'ZdMYuSvrCPq2qv5yQ01wpE6FgaQJwvytuYuNnkuWdoMLpjO0sg'
access_token = '1315688543784562688-UYAqRnekf0mDKoQgsZWoKiBGqlffNw'
access_token_secret = 'wvORx0dHxp9uGXXjAjZEJ1hj8eLGcckRlOlmzvxhSqAEA'
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)
tweets = []
company_twitter_name = ['MemphisMeats','mosa_meat','_superMeat_','ModernMeadow',
                        'MissionBarns','BalleticFoods','CubiqF',
                        'NewAgeMeats','FinlessFoods','wildtypefoods','itsmeatable',
                        'eatGOURMEY','eatjust','shiokmeats','makingmeat','BlueNaluInc','FutureMeat1','AvantMeats',
                        'AlephFarms','biftekco','vowfood','LabFarmFoods']
for company_name in company_twitter_name:
    print(company_name)
    tweets += api.user_timeline(company_name,count=200,page=1,include_rts=False,include_entities=True,tweet_mode='extended')
    for i in range(2,20):
        tweets += api.user_timeline(company_name, count=200, page=i,include_rts=False,tweet_mode='extended')
    time.sleep(10)
tweet_text = []
published_date = []
Company = []
URL = []
for tweet in tweets:
    if tweet.in_reply_to_status_id == None:
        tweet_text.append(tweet.full_text)
        published_date.append(tweet.created_at)
        Company.append(tweet.user.name)
        URL.append(f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}")
dat = pd.DataFrame(data=[text for text in tweet_text],columns=['tweets'])
dat["date"]=np.array([date for date in published_date])
dat["Company"]=np.array([company for company in Company])
dat['url']=np.array([url for url in URL])
dat.to_csv("Raw tweets_December22_10:27PM_CST.csv",mode='a',index=False)



'''---------------------------------------------------------------------------------------
Text cleaning:remove @user, URLs, hashtags, non-alphabets characters and special characters (&amp)
保留lowercase，轉換為root form，去除長度小於4的單詞'''

import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
import pandas as pd
text = pd.read_csv('Raw tweets_December22_10:27PM_CST.csv')
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            #words = words.replace('#','') #remove hashtag symbol
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            token = porter.stem(token) #root form
            doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]




'''---------------------------------------------------------------------------------------
生成bigram：基於gensim.model.phrase的方式'''

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) #將句子變為token，生成list
data_words = list(sent_to_words(text['processed'])) #tokenization
bigram = gensim.models.Phrases(data_words,min_count=1,threshold=1)
'''mincount：兩個單詞共同出現次數小於該值，則不會被考慮為bigram，
threshold：Phrases功能中會生成一個'phase score'，超過這個score的bigram會生成在最終結果中，總的來說，mincount越小，threshold越小，最終生成的bigram越多'''
bigram_mod = gensim.models.phrases.Phraser(bigram) #生成bigram
def make_bigrams(texts): #生成bigram
    return [bigram[doc] for doc in texts]
data_words_bigrams = make_bigrams(data_words)
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]



'''---------------------------------------------------------------------------------------
counting coherence score,使用的是u_mass'''

def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start,limit,step):
        model=gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics,random_state=3)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='u_mass')
        coherence_values.append(coherencemodel.get_coherence())
        i =1
        print(i)
    return model_list, coherence_values
limit=21; start=10; step=1
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=start, limit=limit, step=step)
# Show graph
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values,label='All tweets')
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(loc='best')
plt.xticks(range(start,limit,step))
plt.show()


'''---------------------------------------------------------------------------------------
print bigrams within each topic'''

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=11,random_state=3,alpha='auto',per_word_topics=True)
#pprint(lda_model.print_topics(num_words=400))
for idx, topic in lda_model.show_topics(formatted=False,num_topics=11,num_words= 400):
    print('\n',end='')
    print('Topic:',idx)
    for w in topic:
        if '_' in w[0]:
            print(round(w[1],4),'*',w[0],', ',end='',sep='')


'''---------------------------------------------------------------------------------------
output LDA，生成一個包含dominant topic，keywords，probability of the topic的csv文件
並通過這個文件，統計各個主題內tweets的數量'''

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=text):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: x[1], reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=text['tweets'])
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.to_csv('dominant topics_December_22_10:27PM_CST_data.csv')
data = pd.read_csv('dominant topics_December_22_10:27PM_CST_data.csv')
# tweets count
print('\n')
print('Number of Tweets within each topic:',data.groupby('Dominant_Topic')['Text'].count())

'''---------------------------------------------------------------------------------------
統計Top 20 'hashtags，通過date parameter，以及drop.cuplicates()，除去重複的'hashtag，
也就是說，如果同一個'hashtag在一篇tweet中出現兩次或以上，則只被計算一次'''

import pandas as pd
import numpy as np
import collections
data= pd.read_csv('Raw tweets_December22_10:27PM_CST.csv')
hashtag = []
date = []
for index,i in data.iterrows():
    for word in i['tweets'].split():
        if word.startswith('#'):
            date.append(i['date'])
            hashtag.append(word)
hashtag_trend = pd.DataFrame(data=hashtag,columns=['#'])
hashtag_trend['date'] = np.array(date)
data.drop_duplicates()
counts = collections.Counter(hashtag)
print('\n')
print('Top 20 hasgtags:',counts.most_common(20))

ModuleNotFoundError: No module named 'gensim'